In [ ]:
import os

import numpy as np
import torch
from lib.config import cfg

import lib.dataset.dataset3D as dataUtils
from lib.vortex.vortex import Vortex
from lib.config.project_manager import ProjectManager

project = ProjectManager()
project.load('Ralph_All3D')

#project.create_new(
#    name = 'Ralph_All3D',
#    dataset2D_path = '/home/trackingsetup/Documents/Ralph_All',
#    dataset3D_path = '/home/trackingsetup/Documents/Ralph_All')

cfg = project.get_cfg()

In [ ]:
training_set = dataUtils.VortexDataset3D(cfg, set='train')
val_set = dataUtils.VortexDataset3D(cfg, set='val')
calibPaths = [training_set.coco.dataset['calibration']['intrinsics'], 
              training_set.coco.dataset['calibration']['extrinsics']]

In [ ]:
weights = '/home/trackingsetup/Documents/Vortex/projects/Ralph_All3D/models/vortex/Run_20210929-201946/Vortex-d_60.pth'

efficienttrack_weights = '/home/trackingsetup/Documents/Vortex/projects/Ralph_All3D/models/efficienttrack/Run_20210929-163105/EfficientTrack-d2_40.pth'

vortexNet = Vortex('train', cfg, calibPaths, weights, None)

In [ ]:
vortexNet.train(training_set, val_set, 1000)